<a href="https://colab.research.google.com/github/SerasLain/catandthekittens/blob/master/CAT_collocates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Поиск коллокаций в CAT

In [0]:
# Connection to my Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

# Some useful paths
cat_d = '/content/gdrive/My Drive/CAT/'
cat_together = cat_d + 'Склеенные/'
cat_cleaned = cat_d + 'Cleaned/'
path = '/content/gdrive/My Drive/Новые conll по доменам/NewVers/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## Препроцессинг нового корпуса

In [0]:
import os
import re
import nltk
from nltk.corpus import stopwords


In [0]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
# Connection to my Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Файл с целиковым корпусом проще записать из остальных файлов, чем препроцессить отдельно
cat_d = '/content/gdrive/My Drive/CAT/'
cat_together = cat_d + 'Склеенные/'
files = os.listdir(cat_together)
files.pop(0)


'CAT.txt'

In [0]:
#From https://github.com/MariaFjodorowa/catandthekittens/blob/develop/CYBERsuffering/Preprocessing%20and%20modelling%20example.ipynb
# by Alex Klimov
#относительно хороший результат чистки корпуса
stopwords = ['б', 'г', 'д', 'е', 'ё', 'ж', 'з', 'й', 'л', 'м', 'н', 'п', 'р', 'т', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'гг']
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
goal_d = cat_d + 'Cleaned/'
if not os.path.exists(goal_d):
  os.mkdir(goal_d)
work_d = cat_together
for i, file in enumerate(files):
    text = os.path.abspath(cat_together + file)
    with open(text, 'r', encoding = 'utf-8') as t:
      txt = t.read()  
    txt = txt.lower()
    tokenized_text = tokenizer.tokenize(txt)
    lst = []
    for tryingtoclean in tokenized_text:
        tryingtoclean = re.sub('(данные об авторе)(\.)?.*', '', tryingtoclean, flags=re.S)
        tryingtoclean = re.sub(r'[a-zA-z]', '', tryingtoclean)
        tryingtoclean = re.sub(r'_', '', tryingtoclean)
        tryingtoclean = re.sub(r'\d', '', tryingtoclean)
        tryingtoclean = re.sub(r'[ñâÿçàíšßãóìøþôõýæùúñî¹áîřðûɫɥɨêɜǎäiöüéɨïіïëèåòè]', '', tryingtoclean)
        words = re.findall(r'\w+', tryingtoclean)
        clean = filter(lambda a: a not in stopwords, words)
        x = ' '.join(clean).rstrip()
        lst.append(x)
    xxx = '. '.join(lst).rstrip() + '.'
    dotpattern = re.compile(r'(\. \.)+')
    xxx = re.sub(dotpattern, '', xxx)
    spacepattern = re.compile(r'(  )+')
    xxx = re.sub(spacepattern, '', xxx)
    dotpattern2 = re.compile(r'( \.)+')
    xxx = re.sub(dotpattern2, '', xxx)
# Отсюда убран небольшой кусок кода, поскольку он ошибочно срабатывал на строках типа "однако он", превращая их в "однакн"
    goal_f = open(goal_d + file, 'w', encoding = 'utf-8')
    goal_f.write(xxx)
    goal_f.close()

In [0]:
# Checking out
with open(goal_d + 'Economics.txt') as f:
  t = f.read()
  
print(t[:500])

введение в в российской экономике начался восстановительный рост. однако он остается слабым и неустойчивым при этом сохраняются угрозы новой рецессии статья подготовлена при финансовой поддержке рффи проект санкт петербургский государственный университет вестник санкт петербургского университета. вып. экономика шедших десятилетия россия пережила три циклических спада характерных для среднесрочных колебаний капиталистической экономики. при всех существенных отличиях свойственных этим кризисам в п


In [0]:
# preprocessing


#Склеиваем все тексты в один файл для каждого домена, а потом эти файлы в один общий для всего корпуса.

def make_raws(root=cat_d):
    failed = []
    for d in os.listdir(root):
        with open((os.path.join(root, d) + r'.txt'), 'a', encoding='utf-8') as domain:
            for f in os.listdir(os.path.join(root, d)):
                print(f)
                with open(os.path.join(root, d, f), 'r', encoding='utf-8') as t:
                    try:
                        text = t.read()
                        domain.write(text)
                        domain.write('\n')
                    except UnicodeDecodeError:
                        failed.append(os.path.join(root, d, f))
    print(failed)
    
make_raws()

In [0]:
def make_corpus(root='.'):
    for root, dirs, files in os.walk(root):
        for f in files:
            with open(os.path.join(root, 'CAT.txt'), 'a', encoding='utf8') as corpora:
                with open(os.path.join(root, f), 'r', encoding='utf8', errors='ignore') as f:
                    domain = f.read()
                corpora.write(domain)

In [0]:
make_corpus(root=goal_d)


## Извлечение коллокаций

Версия для старого корпуса https://github.com/MariaFjodorowa/catandthekittens/tree/develop/collocations/collocation_frequencies
by Anna Dmitrieva

Код переработан для лучшей читаемости и оформлен по pep8

In [0]:
pip install conllu

In [0]:
from conllu import parse, parse_tree

In [0]:
def parser(filename):
  """
  Yields a sentence from conllu tree with its tags
  
  """
  """
  >>> for i in parser('/content/gdrive/My Drive/Новые conll по доменам/NewVers/CleanedPsyEdu.conllu'):
      print(i)   
  TokenList<Музыка, звучит, отовсюду, независимо, от, нашего, желания, или, нежелания, слушать, ее, .>
  """
  with open(filename, 'r', encoding='utf-8') as f:
    data = f.read()
  tree = parse(data)
  for token in tree:
    yield token

In [0]:
tree = parser('/content/gdrive/My Drive/Новые conll по доменам/NewVers/CleanedPsyEdu.conllu')

In [0]:
import os
os.listdir('/content/gdrive/My Drive/Новые conll по доменам/NewVers/')

['PsyEdu.xlsx',
 'Domains_collocations',
 'CleanedEco.conllu',
 'CleanedLaw.conllu',
 'CleanedLing.conllu',
 'CleanedPsyEdu.conllu',
 'CleanedSocioHist.conllu',
 'CleanedCAT.conllu',
 'Eco.conllu.xlsx',
 'PsyEdu.csv',
 'PsyEdu_uni.csv',
 'PsyEdu_unigrams.csv',
 'PsyEdu_bi.csv',
 'PsyEdu_bi.csv_tscore.csv']

In [0]:
# Извлекаем слова
def get_words(tree):
  """
  tree - generator of sentences (TokenLists) from conllu tree
  
  words, list is a list of all tokens we need from the tree
  size, int is a number of all words in the domain
  """
  words = []
  for sentence in tree:
    for token in sentence:
      if token['form'] != '_' and token['upostag'] != '_' and token['upostag']!='NONLEX' and token['form'] not in r'[]\/':
        for wordform in token['form'].lower().split():
          words.append((wordform, token['upostag']))
  size = len(words)
  return words, size # arr of tuples

In [0]:
words, size = get_words(tree)

In [0]:
del tree

In [0]:
print(size)

2084647


In [0]:
from collections import defaultdict
def get_ngrams(words, n):
    # arr of tuples, int
    """
    words, list is a list of all tokens we need from the tree
    n, int is a number of words in n-gram, from 1 to 6
    
    ngrams, defaultdict is a dictionary of ngrams
    """
    ngrams = defaultdict(list)
    if n == 1:
      #unigrams is just a dictionary of all tokens with their counting in the domain
        for i in words:
            if i[1] != 'SENT':
              try:
                  ngrams[i][0] += 1
              except IndexError:
                ngrams[i].append(1)
              ngrams[i].append(i[1])
    else:
        for i in range(len(words) - n):
            ngram = words[i:(i + n)]
            for tup in ngram:
                if tup[1]=='SENT':
                    ngram.clear()
                    break
            if ngram:
                word = ''
                tag = ''
                for tup in ngram:
                  # хз, насколько осмысленно собирать их в строку с пробелами, а не в массив
                  word = word + tup[0] + ' '
                  tag = tag + tup[1] + ' '
                try:
                  ngrams[word[:-1]][0] += 1
                except IndexError:
                  ngrams[word[:-1]].append(1)
                ngrams[word[:-1]].append(tag[:-1])
    return ngrams #arr of tuples of two strings

In [0]:
unigrams = get_ngrams(words, 1)


In [0]:
def write_csv(ngrams, path):
  with open(path, 'a', encoding='utf-8') as f:
    for ngram in ngrams:
      f.write('\t'.join([ngram, str(ngrams[ngram][0]), ngrams[ngram][1]])+'\n')

In [0]:
write_csv(trigrams, path+'PsyEdu_tri.csv')

In [0]:
del bigrams

In [0]:
bigrams = {}
def read_ngrams(path, container):
  """
  Load dictionary of ngrams from csv
  """
  """
  path, str is a path to source csv
  container, dictionary is an empty dict which will be filled with collocations
  """
  
  file = open(path, 'r', encoding='utf-8')
  for line in file:
    collocation, freq, tags = line.strip('\n').split('\t')
    container[collocation] = [freq, tags]
  return container

In [0]:
# Слово частота тэг
unigrams = {}
file = open(path+'PsyEdu_unigrams.csv','r', encoding='utf-8')
for line in file:
  print(line)
  word_tag, freq = line.strip('\n').split('\t')
  unigrams[word_tag] = [freq]
  

In [0]:
trigrams = get_ngrams(words, 3)

In [0]:
def raw_freq(colloc_tup, size):
  """
  colloc_tup, tuple is a tuple like ('word another_word', 3, 'Tag another_tag')
  
  rf, float is a raw_frequency
  
  """
  count = colloc_tup[2]
  rf = int(count)/size
  return rf

In [0]:
import math
import numpy as np

def t_score(colloc_tup, unigrams, size):
  """
  colloc_tup, tuple is a tuple like ('word another_word', 'Tag another_tag', 3)
  unigrams, defaultdict is a dict of word:frequency in corpora, frequency is int
  
  score, float is a t-score metric
  counted by formula from http://webground.su/data/lit/pivovarova_yagunova/
  Izvlechenie_i_klassifikatsiya_terminoligicheskih_kollokatsyi.pdf
  """
  collocation = colloc_tup[0].split(' ')
  tags = colloc_tup[1].split(' ')
  collocation_frequency = int(colloc_tup[2])
  collocation_dict = {}
  for i in range(len(collocation)):
    collocation_dict[collocation[i]] = int(unigrams[(collocation[i], tags[i])][0])
  score = (collocation_frequency - float(np.prod(list(collocation_dict.values()))))/math.sqrt(collocation_frequency)
  return score
  
  

In [0]:
def log_dice(colloc_tup, unigrams):
  """
  colloc_tup, tuple is a tuple like ('word another_word', 'Tag another_tag', 3)
  unigrams, defaultdict is a dict of word:frequency in corpora, frequency is int
  
  score, float is a logDice metric, from https://www.fi.muni.cz/usr/sojka/download/raslan2008/13.pdf
  """
  collocation = colloc_tup[0].split(' ')
  tags = colloc_tup[1].split(' ')
  collocation_frequency = int(colloc_tup[2])
  collocation_dict = {}
  for i in range(len(collocation)):
    collocation_dict[collocation[i]] = int(unigrams[(collocation[i], tags[i])][0])
  score = 14 + math.log2(2 * collocation_frequency / sum(collocation_dict.values()))
  return score

In [0]:
# Реализация likelyhood ratio (в процессе разработки)
def L(k, n, x):
  """
  k, n, x is floats or integers
  B, float is a binomial coefficient
  """
  B = math.pow(x, k) * math.pow((1 - x), (n - k))
  return B

def likelyhood_ratio(colloc_tup, unigrams, size):
   """
  colloc_tup, tuple is a tuple like ('word another_word', 'Tag another_tag', 3)
  unigrams, defaultdict is a dict of word:frequency in corpora, frequency is int
  
  score, float is a likelyhood ratio metric, from https://nlp.stanford.edu/fsnlp/promo/colloc.pdf Manning&Shutz
  """  
  p = c2 / corpus_len
  p1 = c12 / c1
  p2 = (c2 - c12) / (corpus_len - c1)
  lik_rat = -2*(math.log(L(c12,c1,p)) + math.log(L(c2 - c12, corpus_len - c1, p)) - math.log(L(c12, c1, p1)) - math.log(L(c2 - c12, corpus_len - c1, p2)))

In [0]:
bi = path+'PsyEdu_bi.csv'
def by_row(collocation_csv):
  """
  Takes a dataset with collocation, frequenvcies and tags to process it row by row and write results in the same way into another dataset
  """
  
  """
  collocation_csv, str is a path to the file with ngrams, counting and tags
  """
  f =  open(collocation_csv, 'r', encoding='utf-8')
  file = open(collocation_csv+'_tscore.csv', 'a', encoding='utf-8')
  for line in f:
    collocation, freq, tags = line.strip('\n').split('\t')
    rf = raw_freq((collocation, tags, freq), size)
    t_sc = t_score((collocation, tags, freq), unigrams, size)
    logD = log_dice((collocation, tags, freq), unigrams)
    file.write('\t'.join([collocation, tags, str(freq), str(rf), str(t_sc), str(logD)]) + '\n')
  file.close()
  f.close()

In [0]:
by_row(bi)

In [0]:
tri = path+'PsyEdu_tri.csv'
by_row(tri)